In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from tqdm import tqdm
import torch.nn.functional as F
from gensim.models import Word2Vec
from sklearn import metrics
from sklearn.metrics import classification_report
import itertools

In [18]:
import torch
import torch.nn as nn

In [19]:
class BertModel(nn.Module):
    def __init__(self, voc_size:int=30000, seq_len: int=512, d_model: int=384, d_ff:int=3072, pad_idx: int=1,
                num_encoder: int=12, num_heads: int=12, n_layers=6, dropout: float=0.1):
        super(BertModel, self).__init__()
        self.pad_idx = pad_idx
        self.layers = nn.ModuleList([nn.TransformerEncoderLayer(d_model=d_model, nhead=8, batch_first=True) for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model)
        self.activ1 = nn.Tanh()
        self.classifier = nn.Linear(d_model, 1)
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        output = input
        for layer in self.layers:
            output = layer(output)
        output = output[0][0].unsqueeze(0)
        h_pooled = self.activ1(self.fc(output)) # [batch_size, d_model]
        logits_clsf = self.classifier(h_pooled) # [batch_size, 2]
        return logits_clsf # [B, S, D_model]

In [20]:
def log_metrics(preds, labels):
    preds = torch.stack(preds)
    preds = preds.cpu().detach().numpy()
    labels = torch.stack(labels)
    labels = labels.cpu().detach().numpy()
    
    
    ## Method 2 using ravel()
    fpr_micro, tpr_micro, _ = metrics.roc_curve(labels.ravel(), preds.ravel())
    auc_micro = metrics.auc(fpr_micro, tpr_micro)
    n=10
    preds = [round(i) for i in preds.ravel()]
    acc = metrics.accuracy_score(labels.ravel(), preds)
    f1 = metrics.f1_score(labels.ravel(), preds)
    preds = [preds[i*n:(i+1)*n] for i in range(len(preds)//6)]
    #target_names = list(genre_list.keys())
    #print(classification_report(labels,preds, target_names=target_names))
    return {"auc_micro": auc_micro, "acc" : acc, "f1" : f1, 'pred' : preds}

In [21]:
device='cuda'

In [22]:
def train_one_epoch(dataloader, model, optimizer, device, loss_fn):
    model.train()
    tk0 = tqdm(dataloader, total=len(dataloader))
    total_loss = 0.0
    
    for source_tensor, label in tk0:
        optimizer.zero_grad()
        output = model(source_tensor.to(device))
        loss = loss_fn(output.view(-1, output.size(-1)), torch.FloatTensor([label]).to(device))
        
        total_loss += loss
        loss.backward()
        optimizer.step()
        
    avg_train_loss = total_loss / len(dataloader) 
    print(" Average training loss: {0:.2f}".format(avg_train_loss))  

def test_one_epoch(testset, model, loss_fn):
    model.eval()
    tk0 = tqdm(testset, total=len(testset))
    targets = []
    outputs = []
    total_loss = 0.0
    with torch.no_grad():
        for source_tensor, label in tk0:
            label = torch.FloatTensor([label])
            output = model(source_tensor.to(device))
            loss = loss_fn(output.view(-1, output.size(-1)), label.to(device))
            total_loss += loss
            targets.extend(label)
            outputs.extend(torch.sigmoid(output[0]).to('cpu'))
    avg_valid_loss = total_loss / len(testset) 
    score = log_metrics(outputs, targets)
    print(" Average valid loss: {0:.2f}".format(avg_valid_loss))  
    print('AUC_SCORE: ', score["auc_micro"], " acc: ", score["acc"], "f1: ", score["f1"])
    return score, avg_valid_loss
    
def fit(model, train_dataloader, valid_dataloader=None,test_dataloader=None, EPOCHS=3, lr=0.000002):
    loss_fn = nn.BCEWithLogitsLoss() #ignore padding
    optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
    for i in range(EPOCHS):
        print(f"EPOCHS:{i+1}")
        print('TRAIN')
        train_one_epoch(train_dataloader, model, optimizer, device, loss_fn)
        print('Valid')
        auc, loss = test_one_epoch(valid_dataloader,model, loss_fn)
        if i == 0:
            torch.save(model,'rnn_best.model')
            best_loss = loss
        if  loss<best_loss:
            best_loss = loss
            print(i,' EPOCH BEST MODEL!')
            torch.save(model,'rnn_best.model')
        print('Test')
        auc, loss = test_one_epoch(test_dataloader,model, loss_fn)
    best_model = torch.load('rnn_best.model')
    auc, loss  = test_one_epoch(test_dataloader,best_model, loss_fn)
    print('BEST MODEL')
    print('AUC_SCORE: ', auc["auc_micro"], " acc: ", auc["acc"], "f1: ", auc["f1"])
    return auc

In [40]:
original = []
reverse = []
path1 = 'word2vec_model_text3/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    
    pre_embs = embs
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,scene_num,384)
    for i,emb in enumerate(embs[0]):
#        if i == 0:
#            continue
        reverse_emb[0][-(i+1)] = emb

    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    
    

In [41]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [42]:
bert = BertModel(d_model=384).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|██████████████████████████████████████| 2000/2000 [00:16<00:00, 117.77it/s]


 Average training loss: 0.68
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 859.90it/s]


 Average valid loss: 0.67
AUC_SCORE:  0.7107000000000001  acc:  0.59 f1:  0.6893939393939394
Test


...

 Average valid loss: 0.71
AUC_SCORE:  0.7613  acc:  0.685 f1:  0.72
Test


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 884.81it/s]


 Average valid loss: 0.57
AUC_SCORE:  0.8243999999999999  acc:  0.74 f1:  0.7592592592592593


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 880.78it/s]

 Average valid loss: 0.56
AUC_SCORE:  0.8082  acc:  0.735 f1:  0.7685589519650655
BEST MODEL
AUC_SCORE:  0.8082  acc:  0.735 f1:  0.7685589519650655
{'auc_micro': 0.8082, 'acc': 0.735, 'f1': 0.7685589519650655, 'pred': [[1, 0, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1], [1, 1, 1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 0, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 0, 1, 0, 0, 1], [0, 1, 0, 1, 1, 0, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 1, 0], [0, 0, 1, 0, 0, 0, 1, 0, 0, 0], [1, 1, 0, 1, 1, 1, 0, 1, 0, 1], [1, 0, 0, 1, 0, 0, 1, 0, 1, 1], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 1, 1, 0, 0], [], [], [], [], [], [], [], [], [], [], [], [], []]}


In [7]:
import random

In [8]:
original = []
reverse = []
path1 = 'word2vec_model_char/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    
    pre_embs = embs
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,scene_num,384)
    for i,emb in enumerate(embs[0]):
#        if i == 0:
#            continue
        reverse_emb[0][-(i+1)] = emb

    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    
    

/tmp/ipykernel_1661273/4096105555.py:31: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484810403/work/torch/csrc/utils/tensor_numpy.cpp:172.)
  emb = torch.from_numpy(emb)


In [9]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [10]:
bert = BertModel(d_model=384).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|███████████████████████████████████████| 2000/2000 [00:27<00:00, 72.04it/s]


 Average training loss: 0.69
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 823.10it/s]


 Average valid loss: 0.66
AUC_SCORE:  0.7131  acc:  0.6 f1:  0.6946564885496184
Test


...

 Average valid loss: 0.71
AUC_SCORE:  0.7667999999999999  acc:  0.66 f1:  0.6991150442477876
Test


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 445.54it/s]


 Average valid loss: 0.66
AUC_SCORE:  0.802  acc:  0.72 f1:  0.7407407407407408


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 511.58it/s]

 Average valid loss: 0.53
AUC_SCORE:  0.8186  acc:  0.73 f1:  0.7476635514018691
BEST MODEL
AUC_SCORE:  0.8186  acc:  0.73 f1:  0.7476635514018691
{'auc_micro': 0.8186, 'acc': 0.73, 'f1': 0.7476635514018691, 'pred': [[1, 1, 0, 1, 1, 0, 0, 1, 0, 1], [0, 0, 1, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 1, 0, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 1], [0, 1, 0, 1, 0, 0, 0, 1, 0, 1], [0, 0, 1, 0, 1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 1, 1, 0], [1, 1, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 1, 1, 0, 1, 0], [0, 0, 0, 1, 0, 0, 0, 0, 1, 1], [0, 1, 1, 0, 1, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 1, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [], [], [], [], [], [], [], [], [], [], [], [], []]}


In [139]:
original = []
reverse = []
path1 = 'word2vec_model_extra+emb/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    
    pre_embs = embs
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,scene_num,384)
    for i,emb in enumerate(embs[0]):
#        if i == 0:
#            continue
        reverse_emb[0][-(i+1)] = emb

    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    
    

In [140]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [141]:
bert = BertModel(d_model=384).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|██████████████████████████████████████| 2000/2000 [00:17<00:00, 114.06it/s]


 Average training loss: 0.69
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 857.94it/s]


 Average valid loss: 0.67
AUC_SCORE:  0.7226  acc:  0.555 f1:  0.6763636363636364
Test


...

 Average valid loss: 0.58
AUC_SCORE:  0.7796  acc:  0.705 f1:  0.7064676616915423
Test


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 849.09it/s]


 Average valid loss: 0.50
AUC_SCORE:  0.8371000000000001  acc:  0.755 f1:  0.7655502392344498
EPOCHS:9
TRAIN


 28%|███████████                            | 568/2000 [00:05<00:12, 113.06it/s]


KeyboardInterrupt: 

In [121]:
original = []
reverse = []
path1 = 'word2vec_model_extra+pos/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    
    pre_embs = embs
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,scene_num,400)
    for i,emb in enumerate(embs[0]):
#        if i == 0:
#            continue
        reverse_emb[0][-(i+1)] = emb

    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    
    

In [124]:
print(len(original))
print(len(reverse))

1202
1202


In [125]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [126]:
bert = BertModel(d_model=400).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|██████████████████████████████████████| 2000/2000 [00:16<00:00, 117.69it/s]


 Average training loss: 0.64
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 848.56it/s]


 Average valid loss: 0.56
AUC_SCORE:  0.8585999999999999  acc:  0.745 f1:  0.7605633802816901
Test


...

 Average valid loss: 0.43
AUC_SCORE:  0.8835999999999999  acc:  0.77 f1:  0.7788461538461539
Test


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 845.97it/s]


 Average valid loss: 0.33
AUC_SCORE:  0.9368000000000001  acc:  0.845 f1:  0.8558139534883721
EPOCHS:6
TRAIN


 40%|███████████████▋                       | 806/2000 [00:06<00:10, 116.14it/s]


KeyboardInterrupt: 

In [148]:
original = []
reverse = []
path1 = 'word2vec_model_edge/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    
    pre_embs = embs
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,400)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.wv.index_to_key:
            emb = w2v.wv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,scene_num,400)
    for i,emb in enumerate(embs[0]):
#        if i == 0:
#            continue
        reverse_emb[0][-(i+1)] = emb

    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    
    

In [149]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [150]:
bert = BertModel(d_model=400).to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|██████████████████████████████████████| 2000/2000 [00:16<00:00, 118.22it/s]


 Average training loss: 0.68
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 812.67it/s]


 Average valid loss: 0.66
AUC_SCORE:  0.6969000000000002  acc:  0.635 f1:  0.6995884773662552
Test


...

 Average valid loss: 0.73
AUC_SCORE:  0.7702  acc:  0.675 f1:  0.7085201793721974
Test


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 845.38it/s]


 Average valid loss: 0.54
AUC_SCORE:  0.8533999999999999  acc:  0.78 f1:  0.7904761904761904


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 845.12it/s]

 Average valid loss: 0.47
AUC_SCORE:  0.8575  acc:  0.785 f1:  0.7981220657276995
BEST MODEL
AUC_SCORE:  0.8575  acc:  0.785 f1:  0.7981220657276995
{'auc_micro': 0.8575, 'acc': 0.785, 'f1': 0.7981220657276995, 'pred': [[1, 1, 1, 1, 1, 1, 0, 1, 0, 1], [1, 0, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 1, 1, 1, 0, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 0, 1, 0, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 0], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 0, 1, 1, 1, 0, 1], [0, 1, 0, 1, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 1, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 1, 0, 1, 0], [1, 0, 1, 1, 0, 0, 1, 0, 0, 0], [0, 1, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 1, 0, 0, 0, 0, 0, 0, 1], [], [], [], [], [], [], [], [], [], [], [], [], []]}


# 평균

In [46]:
import pickle
with open('emb_avg2.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [127]:
original = []
reverse = []
path1 = 'word2vec_model_char/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    if movie_name not in movie_emb.keys():
        continue
    pre_embs = embs
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])

    k = 0

    real_embs = []
    for emb in movie_emb[movie_name]:
        if str(emb[0]) != 'nan':
            real_embs.append(emb)
    embs = torch.FloatTensor(real_embs).unsqueeze(0)

    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,len(embs[0]),384)
    for i,emb in enumerate(embs[0]):
        reverse_emb[0][-(i+1)] = emb
    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    

In [129]:
embs

tensor([[[ 0.0946,  0.4989,  0.2511,  ..., -0.0374,  0.1700,  0.5408],
         [-0.1516,  0.2858,  0.4151,  ...,  0.1799,  0.0222,  0.2225],
         [ 0.1228,  0.0312,  0.2744,  ...,  0.1853,  0.2074,  0.0189],
         ...,
         [ 0.1141,  0.3026,  0.3107,  ..., -0.1369, -0.1973,  0.2539],
         [ 0.3033,  0.3994,  0.3760,  ..., -0.0968,  0.0949,  0.3761],
         [ 0.1520,  0.1358,  0.3401,  ..., -0.1170, -0.0191,  0.3754]]])

In [130]:
reverse_emb

tensor([[[ 0.1520,  0.1358,  0.3401,  ..., -0.1170, -0.0191,  0.3754],
         [ 0.3033,  0.3994,  0.3760,  ..., -0.0968,  0.0949,  0.3761],
         [ 0.1141,  0.3026,  0.3107,  ..., -0.1369, -0.1973,  0.2539],
         ...,
         [ 0.1228,  0.0312,  0.2744,  ...,  0.1853,  0.2074,  0.0189],
         [-0.1516,  0.2858,  0.4151,  ...,  0.1799,  0.0222,  0.2225],
         [ 0.0946,  0.4989,  0.2511,  ..., -0.0374,  0.1700,  0.5408]]])

In [131]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [132]:
bert = BertModel().to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|██████████████████████████████████████| 2000/2000 [00:16<00:00, 118.30it/s]


 Average training loss: 0.68
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 764.86it/s]


 Average valid loss: 0.64
AUC_SCORE:  0.7629  acc:  0.66 f1:  0.6666666666666666
Test


...

 Average valid loss: 0.55
AUC_SCORE:  0.8065  acc:  0.75 f1:  0.7474747474747474
Test


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 841.39it/s]


 Average valid loss: 0.50
AUC_SCORE:  0.8424999999999999  acc:  0.76 f1:  0.7669902912621359
EPOCHS:11
TRAIN


 49%|███████████████████▏                   | 987/2000 [00:08<00:08, 115.60it/s]


KeyboardInterrupt: 

# Sum

In [8]:
import pickle
with open('emb_sum.pickle','rb') as fw:
    movie_emb = pickle.load(fw)

In [20]:
original = []
reverse = []
path1 = 'word2vec_model_char/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    if movie_name not in movie_emb.keys():
        continue
    pre_embs = embs
    
    w2v=Word2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.wv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])

    k = 0

    real_embs = []
    for emb in movie_emb[movie_name]:
        if str(emb[0]) != 'nan':
            real_embs.append(emb)
    embs = torch.FloatTensor(real_embs).unsqueeze(0).squeeze(2)

    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,len(embs[0]),384)
    for i,emb in enumerate(embs[0]):
        reverse_emb[0][-(i+1)] = emb
    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    

In [21]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [22]:
embs.shape

torch.Size([1, 63, 384])

In [23]:
bert = BertModel().to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|██████████████████████████████████████| 2000/2000 [00:16<00:00, 120.97it/s]


 Average training loss: 0.70
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 867.05it/s]


 Average valid loss: 0.69
AUC_SCORE:  0.54135  acc:  0.525 f1:  0.6494464944649446
Test


...

 Average valid loss: 0.83
AUC_SCORE:  0.5546500000000001  acc:  0.52 f1:  0.5428571428571428
Test


100%|████████████████████████████████████████| 184/184 [00:00<00:00, 893.06it/s]


 Average valid loss: 0.72
AUC_SCORE:  0.6759215500945179  acc:  0.6195652173913043 f1:  0.6428571428571429


100%|████████████████████████████████████████| 184/184 [00:00<00:00, 889.46it/s]

 Average valid loss: 0.68
AUC_SCORE:  0.6437854442344046  acc:  0.5706521739130435 f1:  0.6488888888888888
BEST MODEL
AUC_SCORE:  0.6437854442344046  acc:  0.5706521739130435 f1:  0.6488888888888888
{'auc_micro': 0.6437854442344046, 'acc': 0.5706521739130435, 'f1': 0.6488888888888888, 'pred': [[1, 1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 0, 1, 1, 1, 1, 0, 1], [1, 1, 1, 0, 1, 1, 1, 0, 1, 1], [1, 1, 1, 1, 1, 0, 1, 1, 1, 1], [1, 0, 1, 1, 0, 1, 1, 1, 1, 1], [1, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1, 1, 0, 1, 1], [1, 1, 1, 0, 1, 1, 1, 1, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1, 1, 0], [0, 1, 0, 0, 1, 1, 1, 0, 1, 1], [1, 1, 0, 1, 0, 1, 0, 1, 1, 1], [0, 0, 1, 1, 1, 1, 0, 0, 1, 1], [1, 1, 0, 1, 1, 0, 1, 1, 1, 1], [1, 0, 0, 1, 0, 0, 1, 1, 1, 0], [1, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 0, 0], [0, 0, 0, 1, 0, 1, 1, 1, 0, 1], [1, 1, 0, 1, 1, 1, 1, 1, 1, 0], [1, 0, 1, 1], [], [], [], [], [], [], [], [], [], [], []]}


# Doc2Vec

In [134]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [142]:
original = []
reverse = []
path1 = 'doc2vec/'

filenames = os.listdir(path1)
train_data = []
valid_data = []
test_data = []
embs = []
for num, filename in enumerate(filenames):
    movie_name = filename[:-14]
    
    pre_embs = embs
    
    w2v=Doc2Vec.load(path1 + filename)
    base = 'scene_'
    scene_num = 0
    passs = 0
    max_scene = 0
    for node in w2v.dv.index_to_key:
        if base in node:
            scene_num += 1
            if int(node[6:])>max_scene:
                max_scene = int(node[6:])
    embs = torch.zeros(1,scene_num,384)
    k = 0
    for i in range(1,max_scene+1):
        emb_name = base + str(i)
        if emb_name in w2v.dv.index_to_key:
            emb = w2v.dv[emb_name]
            emb = torch.from_numpy(emb)
            embs[0][k] = emb
            k += 1
    if len(embs[0]>5):        
        original.append([embs,[1]])
    reverse_emb = torch.zeros(1,scene_num,384)
    for i,emb in enumerate(embs[0]):
        reverse_emb[0][(-i)] = emb
    if len(reverse_emb[0]>5):
        reverse.append([reverse_emb,[0]])
    
    

In [143]:
train_data = original[:1000]+reverse[:1000]
valid_data = original[1000:1100]+reverse[1000:1100]
test_data = original[1100:1200]+reverse[1100:1200]
random.Random(4).shuffle(train_data)

In [144]:
bert = BertModel().to('cuda')
score = fit(bert, train_data,valid_data,test_data,EPOCHS=15)
print(score)

EPOCHS:1
TRAIN


100%|██████████████████████████████████████| 2000/2000 [00:17<00:00, 114.35it/s]


 Average training loss: 0.70
Valid


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 890.62it/s]


 Average valid loss: 0.70
AUC_SCORE:  0.4998  acc:  0.5 f1:  0.6575342465753424
Test


...

 Average valid loss: 0.70
AUC_SCORE:  0.49970000000000003  acc:  0.5 f1:  0.6644295302013423
Test


100%|████████████████████████████████████████| 200/200 [00:00<00:00, 888.87it/s]


 Average valid loss: 0.70
AUC_SCORE:  0.4998  acc:  0.5 f1:  0.6666666666666666
EPOCHS:3
TRAIN


 15%|█████▋                                 | 294/2000 [00:02<00:15, 112.91it/s]


KeyboardInterrupt: 